In [104]:
import pandas as pd
import numpy as np
import re
from itertools import accumulate

In [125]:
emails = pd.read_csv(r'C:\Users\amrenkumar\Desktop\TestData\full-output1.csv')

In [126]:
emails.shape

(27226, 17)

In [127]:
emails_clean = emails[~emails.interaction_content.isnull()]

In [128]:
emails_clean = emails_clean.interaction_content

In [129]:
emails_clean.drop_duplicates(keep='first', inplace=True)

In [130]:
emails_clean=emails_clean.take(np.random.permutation(len(emails_clean))[:1000])

In [131]:
emails_content = pd.DataFrame(emails_clean)

In [132]:
emails_content['index'] = emails_clean.index

In [133]:
emails_content_clean= emails_content.interaction_content.str.replace("[^a-zA-Z ]", "")

In [134]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amrenkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [135]:
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [136]:
doc_sample = emails_content[emails_content['index'] == 14].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [137]:
processed_docs = emails_content['interaction_content'].map(preprocess)
processed_docs[:10]

25821    [intra, spot, rat, date, usdbid, usdask, usdmi...
944      [intra, spot, rat, date, usdbid, usdask, usdmi...
22653                                             [nlcobi]
13676    [short, term, rate, trade, februari, jani, dri...
12806    [intra, spot, rat, date, usdbid, usdask, usdmi...
21707    [intra, spot, rat, date, usdbid, usdask, usdmi...
25856    [trade, idea, invest, strategi, discuss, rise,...
14297    [trade, idea, invest, strategi, discuss, rise,...
22632    [client, mnemon, map, sale, relat, revenu, com...
6272     [pull, trade, look, know, action, requir, tick...
Name: interaction_content, dtype: object

In [142]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 date
1 eurask
2 eurbid
3 eurmid
4 gbpask
5 gbpbid
6 gbpmid
7 intra
8 rat
9 spot
10 usdask


In [139]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [143]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[10]

[(0, 2),
 (15, 1),
 (16, 233),
 (18, 1),
 (21, 122),
 (25, 1),
 (27, 3),
 (30, 2),
 (35, 1),
 (38, 1),
 (39, 1),
 (46, 1),
 (47, 8),
 (52, 16),
 (53, 1),
 (55, 116),
 (65, 2),
 (67, 1),
 (72, 9),
 (80, 117),
 (82, 1),
 (92, 2),
 (103, 1),
 (115, 1),
 (118, 2),
 (119, 1),
 (122, 1),
 (127, 1),
 (137, 3),
 (138, 3),
 (141, 36),
 (168, 6),
 (183, 12),
 (187, 1),
 (190, 1),
 (202, 3),
 (207, 2),
 (210, 1),
 (229, 3),
 (256, 10),
 (277, 3),
 (282, 1),
 (288, 1),
 (305, 1),
 (314, 1),
 (323, 1),
 (333, 2),
 (334, 10),
 (339, 76),
 (342, 116),
 (363, 5),
 (366, 1),
 (372, 1),
 (378, 1),
 (387, 1),
 (389, 2),
 (412, 1),
 (417, 1),
 (432, 10),
 (436, 1),
 (445, 7),
 (450, 1),
 (453, 1),
 (454, 1),
 (457, 6),
 (468, 1),
 (476, 4),
 (485, 4),
 (488, 7),
 (498, 116),
 (500, 1),
 (518, 1),
 (547, 4),
 (552, 2),
 (553, 1),
 (555, 3),
 (574, 22),
 (581, 1),
 (595, 1),
 (615, 1),
 (627, 120),
 (629, 1),
 (653, 1),
 (662, 10),
 (696, 14),
 (697, 121),
 (708, 1),
 (709, 2),
 (716, 1),
 (719, 1),
 (723, 

In [144]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.17938968373469746),
 (1, 0.3007455041990572),
 (2, 0.3007455041990572),
 (3, 0.3007455041990572),
 (4, 0.3007455041990572),
 (5, 0.3007455041990572),
 (6, 0.3007455041990572),
 (7, 0.29632462905370255),
 (8, 0.16541872759330464),
 (9, 0.19651202376357674),
 (10, 0.3007455041990572),
 (11, 0.3007455041990572),
 (12, 0.3007455041990572)]


In [145]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [146]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.017*"lynch" + 0.013*"merril" + 0.012*"bank" + 0.010*"sierrdbo" + 0.009*"report" + 0.009*"employe" + 0.009*"rminoo" + 0.007*"expens" + 0.007*"secur" + 0.007*"compani"
Topic: 1 
Words: 0.142*"quot" + 0.021*"bank" + 0.009*"trade" + 0.007*"merril" + 0.007*"payment" + 0.007*"true" + 0.006*"lynch" + 0.006*"bfix" + 0.005*"port" + 0.005*"rat"
Topic: 2 
Words: 0.014*"merril" + 0.013*"europ" + 0.012*"lynch" + 0.009*"trade" + 0.008*"bank" + 0.007*"financi" + 0.007*"price" + 0.006*"report" + 0.006*"unit" + 0.006*"bofa"
Topic: 3 
Words: 0.015*"merril" + 0.014*"lynch" + 0.010*"secur" + 0.009*"bank" + 0.009*"march" + 0.008*"flow" + 0.008*"rate" + 0.008*"real" + 0.007*"america" + 0.007*"money"
Topic: 4 
Words: 0.026*"merril" + 0.026*"lynch" + 0.015*"report" + 0.010*"research" + 0.010*"rat" + 0.010*"bofa" + 0.009*"global" + 0.008*"bank" + 0.008*"financi" + 0.008*"secur"
Topic: 5 
Words: 0.048*"bank" + 0.019*"icap" + 0.015*"date" + 0.015*"unit" + 0.013*"rat" + 0.012*"state" + 0.011*"s

In [224]:
df_annotated_final_clean

,Sent,Token,POS,Tag
0,Sentence:0,E,RB,O
1,Sentence:0,11,CD,O
2,Sentence:0,Sep19,NNP,O
3,Sentence:0,C,NNP,O
4,Sentence:0,132.20,CD,O
5,Sentence:0,S,NNP,O
6,Sentence:0,160.00,CD,O
7,Sentence:0,Brazil,NNP,B-geo
8,Sentence:0,receive,JJ,O
9,Sentence:0,Jan16,NNP,O
